# MongoDB CRUD Statements

- documentation - [https://www.mongodb.com/docs/manual/crud/](https://www.mongodb.com/docs/manual/crud/)
- 
## Create Database, Collection, Documents
- connect to your Atlas server
- create database, collection, and documents

In [1]:
from pymongo import MongoClient
from pymongo.server_api import ServerApi

In [27]:
from pymongo.errors import OperationFailure

In [2]:
path_to_certificate = 'python/x509-cert-MongoDB-Atlas.pem'

In [3]:
uri = "mongodb+srv://cluster0.xeszaub.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"

In [4]:
client = MongoClient(uri,
                     tls=True,
                     tlsCertificateKeyFile=path_to_certificate,
                     server_api=ServerApi('1'))

In [5]:
# Create database called myDatabase
db = client.myDatabase

In [6]:
# create a collection named 'recipes'
# collection is equivalent to table
my_collection = db.recipes

In [7]:
# drop collection in case it already exists
try:
    my_collection.drop()
except pymongo.errors.OperationFailure as ex:
    print("An authentication error was received. Are you sure your uri/certificate is correct?")
    raise ex

In [53]:
help(my_collection)

Help on Collection in module pymongo.collection object:

class Collection(pymongo.common.BaseObject, typing.Generic)
 |  Collection(database: 'Database[_DocumentType]', name: 'str', create: 'Optional[bool]' = False, codec_options: 'Optional[CodecOptions[_DocumentTypeArg]]' = None, read_preference: 'Optional[_ServerMode]' = None, write_concern: 'Optional[WriteConcern]' = None, read_concern: 'Optional[ReadConcern]' = None, session: 'Optional[ClientSession]' = None, **kwargs: 'Any') -> 'None'
 |
 |  A Mongo collection.
 |
 |  Method resolution order:
 |      Collection
 |      pymongo.common.BaseObject
 |      typing.Generic
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __bool__(self) -> 'NoReturn'
 |
 |  __call__(self, *args: 'Any', **kwargs: 'Any') -> 'NoReturn'
 |      This is only here so that some API misusages are easier to debug.
 |
 |  __eq__(self, other: 'Any') -> 'bool'
 |      Return self==value.
 |
 |  __getattr__(self, name: 'str') -> 'Collection[_DocumentType]'

## Insert One Document

- let's insert one document into our recipe collection
- document is equivalent to row/record in SQL databases

In [10]:
elotes = {"name": "elotes", "ingredients": ["corn", "mayonnaise", "cotija cheese", "sour cream", "lime"], "prep_time": 35}

In [13]:
# if the insert_one API runs successfully, double check on Atlas to see the recipe
try:
    my_collection.insert_one(elotes)
except OperationFailure as ex:
    print(ex)
    raise ex
    

## Insert Many Documents

- let's create a list of python dictionaries as documents

In [37]:
recipes = []

In [38]:
loco = { "name": "loco moco", "ingredients": ["ground beef", "butter", "onion", "egg", "bread bun", "mushrooms"], "prep_time": 54 }

In [39]:
recipes.append(loco)

In [40]:
recipes

[{'name': 'loco moco',
  'ingredients': ['ground beef',
   'butter',
   'onion',
   'egg',
   'bread bun',
   'mushrooms'],
  'prep_time': 54}]

In [41]:
patatas =  { "name": "patatas bravas", "ingredients": ["potato", "tomato", "olive oil", "onion", "garlic", "paprika"], "prep_time": 80 }

In [42]:
recipes.append(patatas)

In [43]:
fried_rice = { "name": "fried rice", "ingredients": ["rice", "soy sauce", "egg", "onion", "pea", "carrot", "sesame oil"], "prep_time": 40 }

In [44]:
recipes.append(fried_rice)

In [45]:
recipes

[{'name': 'loco moco',
  'ingredients': ['ground beef',
   'butter',
   'onion',
   'egg',
   'bread bun',
   'mushrooms'],
  'prep_time': 54},
 {'name': 'patatas bravas',
  'ingredients': ['potato',
   'tomato',
   'olive oil',
   'onion',
   'garlic',
   'paprika'],
  'prep_time': 80},
 {'name': 'fried rice',
  'ingredients': ['rice',
   'soy sauce',
   'egg',
   'onion',
   'pea',
   'carrot',
   'sesame oil'],
  'prep_time': 40}]

In [46]:
try:
    my_collection.insert_many(recipes)
except OperationFailure as ex:
    raise ex

## Read/Search/Find Document

### Find all
- retrieve all the documents with `find()` with an empty filter

In [47]:
result = my_collection.find()

In [48]:
if result:
    for doc in result:
        recipe = doc['name']
        ing_count = len(doc['ingredients'])
        prep_time = doc['prep_time']
        print(f'{recipe} has {ing_count} ingredients and takes {prep_time} minutes to make.')
else:
    print('No recipes found!')

elotes has 5 ingredients and takes 35 minutes to make.
loco moco has 6 ingredients and takes 54 minutes to make.
loco moco has 6 ingredients and takes 54 minutes to make.
patatas bravas has 6 ingredients and takes 80 minutes to make.
fried rice has 7 ingredients and takes 40 minutes to make.


### Find One Document

- use find_one API with dictionary as filter - `find_one({<filter>})`

In [49]:
# find a single document that has string 'potato' in the ingredient list
pot_recipe = my_collection.find_one({"ingredients": "potato"})

In [50]:
if pot_recipe is not None:
    print("A recipe which uses potato:")
    print(pot_recipe)
else:
    print("No recipes found that contain 'potato' as an ingredient.")

A recipe which uses potato:
{'_id': ObjectId('65246969eff7a7b93ceedc0f'), 'name': 'patatas bravas', 'ingredients': ['potato', 'tomato', 'olive oil', 'onion', 'garlic', 'paprika'], 'prep_time': 80}


## Update Documents

- You can update a single document or multiple documents in a single call
- Note the 'new=True' option: if omitted, `find_one_and_update()` returns the original document instead of the updated one
- let's change the prep time of the recipe with 'potato' as an ingredient to 72

In [51]:
updated = my_collection.find_one_and_update({'ingredients': 'potato'}, {'$set': {'prep_time': 72}}, new=True)

In [52]:
if updated:
    print("Here's the updated recipe:")
    print(updated)
else:
    print("No recipes found that contain 'potato' as an ingredient.")

Here's the updated recipe:
{'_id': ObjectId('65246969eff7a7b93ceedc0f'), 'name': 'patatas bravas', 'ingredients': ['potato', 'tomato', 'olive oil', 'onion', 'garlic', 'paprika'], 'prep_time': 72}


## Delete Documents

- can delete a single document of all documents that match a specified filter
- to delete all the docuemnts in a collection, pass an emtpy filter to the `delete_many()` API
- let's delete documents in which name field is either 'elotes' or 'fried rice'
- we should delete 2 documents that match the filter

In [54]:
result = my_collection.delete_many({"$or": [{"name": "elotes"}, {"name": "fried rice"}]})

In [56]:
print(f'{result.deleted_count} records deleted!')

2 records deleted!


## MongoDB Query Operators

- [See docs](https://www.w3schools.com/mongodb/mongodb_query_operators.php)